In [ ]:
import pyneuralnet as nn
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
from typing import List, Any
np.random.seed(123)

In [ ]:
class Data(nn.abstract.DataSource2D):
    _train_in : List[Any]
    _train_out : List[Any]

    def __init__(self):
        super().__init__()
        train_in, _ = pickle.load(open('data/mnist_preprocessed.pickle', 'rb'))
        self._train_in = [np.reshape(x, (28,28,1)).astype(np.float32) for x in train_in]
        self._train_out = [np.reshape(x, (28,28,1)).astype(np.float32) for x in train_in]

    def size(self):
        return len(self._train_in)

    def sample(self, i : int):
        return (self._train_in[i], self._train_out[i])

In [ ]:
N = 10
a = 0.00001

In [ ]:
model = nn.compound.Sequence(
    nn.compound.UNet(1, 0.6, True),
    nn.convolution.Conv2D(2, 1),
    nn.convolution.Sigmoid2D()
)
# model = nn.compound.Sequence(
#     nn.convolution.Conv2D(1,3,0.9,0.999),
#     nn.normalize.BatchRenorm2D(1,0.8,0.9,0.999),
#     nn.convolution.ReLU2D(),
#     nn.convolution.Conv2D(3,1,0.9,0.999),
#     nn.normalize.BatchRenorm2D(1,0.8,0.9,0.999),
#     nn.convolution.ReLU2D(),
# )
# model = nn.compound.UNet(1, 0.8, 0.9, 0.999, True)
data = Data()
loss = nn.loss.L2()
trainer = nn.training.Trainer2D(model, data, loss)

In [ ]:
sample = data.sample(0)

In [ ]:
sample[0].shape

In [ ]:
plt.imshow(sample[0])
plt.colorbar()

In [ ]:
plt.imshow(sample[1])
plt.colorbar()
sample[1].shape

In [ ]:
out = model.forward(sample[0])

In [ ]:

plt.imshow(out)
plt.colorbar()
out.shape

In [ ]:
b = model.backward(out - sample[1])
plt.imshow(b)
plt.colorbar()
b.shape

In [ ]:
model.train()

In [ ]:
ts = time.time()
errors = trainer.train(N, a)
duration = time.time() - ts
print(f"Training of model complete in {duration / N} seconds per epoch, {duration * 1000 / (N * data.size())} milliseconds per sample.")

plt.title("Training Error")
plt.plot(range(len(errors)), errors)
plt.show()

In [ ]:
sample = data.sample(10)

In [ ]:
plt.imshow(sample[0])
plt.colorbar()

In [ ]:
out = model.forward(sample[0])
out.shape

In [ ]:
plt.imshow(out)
plt.colorbar()
print(f"L2 error: {np.square(out - sample[1]).sum() / out.size}")

In [ ]:
import pickle

bts = pickle.dumps(model)
sour_model = pickle.loads(bts)
out2 = sour_model.forward(sample[0])
diff = abs(out - out2)
print(f"{np.min(diff)} - {np.mean(diff)} - {np.max(diff)}")

In [ ]:
plt.imshow(out2)
plt.colorbar()